In [ ]:
#importing the libraries
import re
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop,Adam
from keras import optimizers

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from keras import callbacks
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
import os
from keras.models import Model,load_model
import json
from keras.models import model_from_json, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
chosen_image_size = 100 #image size
total_sample_size = 11000 # total number of pairs to be created


folder_count = 131 #number of folders or classes
image_count = 20 #number of images to be taken from a folder

path =  os.path.join('xxxxx') #defining the path of the training dataset
print(path)

In [ ]:
#defining the functions we'll need for training
def read_image(filename, byteorder='>'):

    with open(filename, 'rb') as f:

        buffer = f.read()

    header, width, height, maxval = re.search(
        b"(^P5\s(?:\s*#.*[\r\n])*"
        b"(\d+)\s(?:\s*#.*[\r\n])*"
        b"(\d+)\s(?:\s*#.*[\r\n])*"
        b"(\d+)\s(?:\s*#.*[\r\n]\s)*)", buffer).groups()

    return np.frombuffer(buffer,
                            dtype='u1' if int(maxval) < 256 else byteorder+'u2',
                            count=int(width)*int(height),
                            offset=len(header)
                            ).reshape((int(height), int(width)))


def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

def compute_accuracy(predictions, labels):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return labels[predictions.ravel() < 0.5].mean()

def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))


In [ ]:
import matplotlib.pyplot as plt
img=plt.imread(path+'S1/1_100.jpg')

In [ ]:
#reading the data and making the pairs
import numpy as np
def get_data(size, total_sample_size):
    img = plt.imread('xxx') #reading the image path
    if resize == True:
        image = image[::size, ::size]
    dim1 = image.shape[0]
    dim2 = image.shape[1]

    count = 0

    #initialize the numpy array with the shape of [total_sample, no_of_pairs, dim1, dim2]
    x_geuine_pair = np.zeros([total_sample_size, 2, 1, dim1, dim2])

    y_genuine = np.zeros([total_sample_size,1])

    for i in range(folder_count):
        for j in range(int(total_sample_size/folder_count)):
            ind1 = 0
            ind2 = 0

            #read images from same directory (genuine pair)
            while ind1 == ind2:
                ind1 = np.random.randint(image_count)
                ind2 = np.random.randint(image_count)

            # read the two images
            image1 = plt.imread(path+'S' + str(i+1) + '/' + str(ind1 + 1) + '_100.jpg', 'rw+')

            image2 = plt.imread(path+'S' + str(i+1) + '/' + str(ind2 + 1) + '_100.jpg', 'rw+')


            #reduce the size
            if resize == True:
                img1 = img1[::size, ::size]
                img2 = img2[::size, ::size]

            #store the images to the initialized numpy array
            print
            x_geuine_pair[count, 0, 0, :, :] = img1
            x_geuine_pair[count, 1, 0, :, :] = img2

            #as we are drawing images from the same directory we assign label as 1. (genuine pair)
            y_genuine[count] = 1
            count += 1

    count = 0
    x_imposite_pair = np.zeros([total_sample_size, 2, 1, dim1, dim2])
    y_imposite = np.zeros([total_sample_size, 1])

    for i in range(int(total_sample_size/image_count)):
        for j in range(image_count):

            #read images from different directory (imposite pair)
            while True:
                ind1 = np.random.randint(folder_count)
                ind2 = np.random.randint(folder_count)
                if ind1 != ind2:
                    break
            image1 = plt.imread(path+'S' + str(ind1+1) + '/' + str(j + 1) + '_100.jpg', 'rw+')

            image2 = plt.imread(path+'S' + str(ind2+1) + '/' + str(j + 1) + '_100.jpg', 'rw+')

            if resize == True:
                img1 = img1[::size, ::size]
                img2 = img2[::size, ::size]

            x_imposite_pair[count, 0, 0, :, :] = img1
            x_imposite_pair[count, 1, 0, :, :] = img2
            #as we are drawing images from the different directory we assign label as 0. (imposite pair)
            y_imposite[count] = 0
            count += 1

    #now, concatenate, genuine pairs and imposite pair to get the whole data
    #print(x_geuine_pair.shape)
    #print(x_imposite_pair.shape)
    X = np.concatenate([x_geuine_pair, x_imposite_pair], axis=0)/255
    Y = np.concatenate([y_genuine, y_imposite], axis=0)

    return X, Y
X, Y = get_data(size, total_sample_size)

In [ ]:
#plotting a plot
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
sns.countplot(Y[:,0])
plt.show()

In [ ]:
#defining a sequential model which serves as a sequential cnn in siamese network and returns a feature vector
import tensorflow as tf
from keras import backend as K
K.set_image_data_format('channels_first')
def build_base_network(input_shape):

    seq = tf.keras.Sequential()
    nb_filter = [16, 32, 16]
    kernel_size = 2
    seq.add(Convolution2D(nb_filter[0], kernel_size, kernel_size, input_shape=input_shape,padding='valid'))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    seq.add(Dropout(.25))
    seq.add(Convolution2D(nb_filter[1], kernel_size, kernel_size, padding='valid'))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    seq.add(Dropout(.25))
    seq.add(Convolution2D(nb_filter[2], kernel_size, kernel_size, padding='valid'))
    seq.add(Activation('relu'))
    seq.add(MaxPooling2D(pool_size=(2, 2)))
    seq.add(Dropout(.25)
    seq.add(Flatten())
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(50, activation='relu'))
    return seq

In [ ]:
input_dim = X.shape[2:]
img_a = Input(shape=input_dim)
img_b = Input(shape=input_dim)

In [ ]:
base_network = build_base_network(input_dim)
feat_vecs_a = base_network(img_a)
feat_vecs_b = base_network(img_b)

In [ ]:
#defining the distance layer
distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([feat_vecs_a, feat_vecs_b])

In [ ]:
#defining optimizer
epochs = 20
rms = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)#RMSprop()
rms = RMSprop()

In [ ]:
#definign the log and model directory
LogDir="xxxxx"
ModelDir="yyyyy"

In [ ]:
# Save results
from tensorflow import keras
csv_logger = keras.callbacks.CSVLogger(LogDir + "/" + LogPath + "-acc.csv", append = True)

entirecheckpointLast = keras.callbacks.ModelCheckpoint(filepath = ModelDir + "/" + LogPath + "-entire-last.h5", verbose = 0)

entirecheckpointBest = keras.callbacks.ModelCheckpoint(filepath = ModelDir + "/" + LogPath + "-entire-best.h5", verbose = 1, save_best_only = True)

In [ ]:
#compiling the model
model = Model([img_a, img_b], distance)
model.compile(loss=contrastive_loss, optimizer=rms,metrics=[accuracy])
model.summary()

In [ ]:
#training the model
img_1 = X[:, 0]
img2 = X[:, 1]
img_1.shape
history = model.fit([img_1, img2], Y, validation_split=.1,
      batch_size= batch_size, verbose=1,epochs=epochs,callbacks = [csv_logger, entirecheckpointLast, entirecheckpointBest,callback_early_stop_reduceLROnPlateau])


In [ ]:
#saving the model
model.save_weights('one_shot_1_2.h5')
with open('model_architecture_1_2.json', 'w') as f:
    f.write(model.to_json())
print('saved')